In [43]:
import tkinter as tk
import pickle
from tkinter import messagebox

# Model
class Plan:
    def __init__(self, plan_type, price):
        self.plan_type = plan_type
        self.price = price

class IndividualClass:
    def __init__(self, class_name, price, date):
        self.name = class_name
        self.price = price
        self.date = date
        
class Student:
    def __init__(self, name, balance, cpf, age):
        #Info
        self.name = name
        self.balance = balance
        self.cpf = cpf
        self.age = age
        
        #Plans/Class
        self.plan = None
        self.individual_classes = []

    def purchase_plan(self, plan):
        if self.balance >= plan.price:
            self.plan = plan
            self.balance -= plan.price
            return True
        return False
    
    def purchase_class(self, indiv_class):
        if self.balance >= indiv_class.price:
            self.balance -= indiv_class.price
            self.individual_classes.append(indiv_class)
            return True
        return False

class Login:
    def __init__(self, username, password, login_type):
        self.username = username
        self.password = password
        self.login_type = login_type



# View

class LoginScreen(tk.Frame):
    def __init__(self, master=None):
        self.master = master
        master.title("Login")

        self.username_label = tk.Label(root, text="Username:")
        self.username_label.grid(row=0, column=0, padx=10, pady=10)

        self.username_entry = tk.Entry(root)
        self.username_entry.grid(row=0, column=1, padx=10, pady=10)

        self.password_label = tk.Label(root, text="Password:")
        self.password_label.grid(row=1, column=0, padx=10, pady=10)

        self.password_entry = tk.Entry(root, show="*")
        self.password_entry.grid(row=1, column=1, padx=10, pady=10)

        self.login_button = tk.Button(root, text="Login", command=self.autenticate_login)
        self.login_button.grid(row=2, column=0, pady=10)

        self.create_button = tk.Button(root, text="Create Account", command=self.create_account)
        self.create_button.grid(row=2, column=1, pady=10)

        

    def create_account(self):

        with open('logins.bin', 'rb+')  as f:
            list_logins = pickle.load(f)

        username = self.username_entry.get()
        password = self.password_entry.get()
        login_type = 'student'

        data = Login(username, password, login_type)

        list_logins.append(data)

        with open('logins.bin', 'wb')  as f:
            pickle.dump(list_logins, f)

        self.master.withdraw()
        messagebox.showinfo("Criar Conta", "Conta criada com sucesso")
        self.master.deiconify()


    def autenticate_login(self):
        list_logins = []

        with open('logins.bin', 'rb+')  as f:
            list_logins = pickle.load(f)

        username = self.username_entry.get()
        password = self.password_entry.get()

        logged = False

        for i in list_logins:
            if i.username == username and i.password == password:
                self.master.withdraw()
                messagebox.showinfo("Login ok", "Você realizou login como {}".format(i.login_type))
                self.master.destroy()
                logged = True


        if not(logged):
            self.master.withdraw()
            messagebox.showinfo("Login falhou", "Não foi possível realizar o Login")
            self.master.deiconify()

        


class StudentView:
    @staticmethod
    def display_student_info(student):
        print("Nome: {}".format(student.name))
        print("Saldo: {}".format(student.balance))
        print("CPF: {}".format(student.cpf))
        print("Idade: {}".format(student.age))

        if student.plan:
            print("Plano: {}".format(student.plan.plan_type))
        else:
            print("Sem plano atualmente.")
        
        if len(student.individual_classes)>0:
            for classes in student.individual_classes:
                print('Aluno têm aula individual no dia {} de {}'.format(classes.date, classes.name))


class TrainerView:
    @staticmethod
    def display_student_plan(student):
        if student.plan:
            print("Aluno {} tem o plano {}".format(student.name, student.plan.plan_type))
        else:
            print("Aluno {} não tem um plano".format(student.name))

        if len(student.individual_classes)>0:
            for classes in student.individual_classes:
                print('Aluno {} têm {} no dia {}'.format(student.name, classes.name, classes.date))




# Controller
class StudentController:
    def __init__(self, student, student_view):
        self.student = student
        self.student_view = student_view
    
    def update_balance(self, amount):
        self.student.balance += amount
    
    def purchase_plan(self, plan):
        result = self.student.purchase_plan(plan)
        if result:
            print("Compra de plano realizada com sucesso.")
        else:
            print("Saldo insuficiente para a compra do plano.")
    
    def purchase_class(self, indiv_class):
        result = self.student.purchase_class(indiv_class)
        if result:
            print("Compra de aula realizada com sucesso.")
        else:
            print("Saldo insuficiente para a compra da aula.")

    def edit_profile(self, student):
        while True:
            user_input = input("Digite o nome que deseja: ")
            if isinstance(user_input, str):
                student.name = user_input
                break
            print("Você não digitou um nome válido")

        while True:
            user_input = input("Digite o novo cpf (string): ")
            if isinstance(user_input, str):
                student.cpf = user_input
                break
            print("Você não digitou um nome válido")
        
        while True:
            user_input = input("Digite sua idade: ")

            try:
                user_input = int(user_input)
            except:
                pass

            if isinstance(user_input, int):
                student.age = user_input
                break
            print("Você não digitou uma idade válida")
        
    def show_student_info(self):
        self.student_view.display_student_info(self.student)


class TrainerController:
    def __init__(self, student, trainer_view):
        self.student = student
        self.trainer_view = trainer_view
    
    def show_student_plan(self):
        self.trainer_view.display_student_plan(self.student)



if __name__ == '__main__':

    ##Instancio alguns logins de exemplo
    usernames = ['Artur Simon', 'Arthur Vargas', 'Alberto Borsatto', 'Anderson Anan', 'Thiago Dias']
    passwords = ['teste1', 'teste2','teste3', 'teste4','teste5']
    login_types = ['user', 'user', 'trainer', 'trainer', 'manager']

    users = []
    for i, x, y in zip(usernames, passwords, login_types):
        users.append(Login(i, x, y))

    #Crio arquivo de logins
    with open('logins.bin', 'wb')  as f:
        pickle.dump(users, f)

    #Alguns planos e aulas para testes
    annual_plan = Plan("Anual", 1000)
    monthly_plan = Plan("Mensal", 150)
    classe1 = IndividualClass('Aula de zumba', 80, '07/02/2023')
    classe2 = IndividualClass('Aula de tango', 100, '08/02/2023')

    #Instancio aluno (modelo), studentview (visão) e student_controller (controlador)
    aluno = Student("Artur Simon", 500, '012.012.012-12', 27)
    student_view = StudentView()
    student_controller = StudentController(aluno, student_view)
    
    #Inicio pela tela de login e rodo o app.
    root = tk.Tk()
    app = LoginScreen(root)
    root.mainloop()
